In [9]:
import os
import numpy as np
import pickle

In [10]:
NAIVE_PATH = 'naive_pths.pkl'
RRT_PATH = 'rrt_pths.pkl'
SW_PATH = 'sw_pths.pkl'
ASTAR_PATH = 'astar_pths.pkl'

In [11]:
def loadFromFile(file):
    f = open(file, 'rb')
    data = pickle.load(f)
    return data

In [12]:
naive = loadFromFile(NAIVE_PATH)
rrt = loadFromFile(RRT_PATH)
sw = loadFromFile(SW_PATH)
astar = loadFromFile(ASTAR_PATH)

In [39]:
total = len(naive)

In [42]:
sameasrrt = 0
sameasnaive = 0
sameasastar = 0
for n, r, s, a in zip(naive, rrt, sw, astar):
    if s == a:
        sameasastar += 1     
        print(len(s), len(a))

3 3
4 4
7 7
3 3
3 3
5 5
7 7
5 5
4 4
4 4
4 4
5 5
6 6
10 10
7 7
7 7
8 8
12 12
8 8
8 8
8 8
9 9
10 10
9 9
9 9
10 10
11 11
10 10
10 10
11 11
14 14
12 12
11 11
12 12
14 14
12 12
12 12
14 14
13 13
14 14
16 16
4 4
5 5
6 6
7 7
3 3
4 4
5 5
8 8
3 3
3 3
3 3
5 5
6 6
9 9
4 4
4 4
4 4
5 5
6 6
12 12
5 5
6 6
8 8
9 9
6 6
6 6
7 7
8 8
9 9
10 10
12 12
7 7
7 7
7 7
8 8
10 10
8 8
8 8
9 9
11 11
9 9
10 10
12 12
11 11
10 10
10 10
10 10
10 10
13 13
12 12
11 11
11 11
14 14
12 12
12 12
12 12
12 12
14 14
13 13
13 13
14 14
14 14
15 15
3 3
3 3
4 4
3 3
4 4
7 7
10 10
3 3
3 3
3 3
3 3
4 4
5 5
4 4
4 4
4 4
4 4
13 13
5 5
9 9
10 10
6 6
10 10
11 11
7 7
9 9
11 11
12 12
8 8
9 9
10 10
3 3
3 3
4 4
9 9
13 13
3 3
5 5
4 4
3 3
3 3
3 3
3 3
4 4
7 7
4 4
8 8
5 5
5 5
9 9
11 11
6 6
6 6
6 6
7 7
10 10
7 7
7 7
8 8
11 11
8 8
8 8
9 9
9 9
9 9
9 9
9 9
10 10
10 10
10 10
11 11
14 14
11 11
12 12
13 13
13 13
14 14
4 4
3 3
4 4
3 3
4 4
5 5
4 4
3 3
3 3
3 3
4 4
5 5
5 5
4 4
4 4
4 4
5 5
9 9
5 5
6 6
7 7
10 10
6 6
7 7
10 10
8 8
8 8
9 9
13 13
8 8
9 9
10 10
12 1

In [41]:
print(sameasrrt / total)
print(sameasnaive / total)
print(sameasastar / total)

0.12563500683179765
0.29411764705882354
0.29411764705882354
